In [1]:
import base64
import os

import cv2
import numpy as np
import requests
import pickle

url = 'http://localhost:8080/api/gethog'

class_brand = ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce',
               'Swift', 'Tata Safari', 'Toyota Innova']

def img2HOG(img):
    v,buffer = cv2.imencode('.jpg',img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url,json={"img":data})
    
    return response.json()

def readData(path):
    response = []
    for folder in os.listdir(path):
        for img_name in os.listdir(path+'/'+folder):
            img_path = path+'/'+folder+'/'+img_name
            img = cv2.imread(img_path)
            res = img2HOG(img)
            hog = list(res["hog"])
            hog.append(class_brand.index(folder))
            response.append(hog)
    
    return response

In [2]:
path = r'Cars Dataset\\train'
datasetHOG = readData(path)

In [3]:
write_path = "imgHOG.pkl"
pickle.dump(datasetHOG,open(write_path,"wb"))

In [4]:
imgHog = pickle.load(open('imgHog.pkl','rb'))
print(len(imgHog))

3352


In [5]:
imgHog_np = np.array(imgHog)
x_train = imgHog_np[:,0:-1]
y_train = imgHog_np[:,-1]

In [6]:
print(x_train.shape)
print(y_train.shape)

(3352, 8100)
(3352,)


In [7]:
path_test = r'Cars Dataset\test'
dataset_test = readData(path_test)

In [8]:
write_path = "imgHOG_test.pkl"
pickle.dump(dataset_test,open(write_path,"wb"))

In [9]:
dataset_test = pickle.load(open('imgHOG_test.pkl','rb'))
print(len(dataset_test))

813


In [10]:
dataset_test_np = np.array(dataset_test)
x_test = dataset_test_np[:,0:-1]
y_test = dataset_test_np[:,-1]

In [11]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [12]:
from sklearn import metrics

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test,y_pred)
cofus_mect = metrics.confusion_matrix(y_test,y_pred)
print("accuracy = ",acc*100)
print('Confusion Mectrix :\n',cofus_mect)

accuracy =  46.74046740467405
Confusion Mectrix :
 [[ 71  14   9  29  29  24  23]
 [  9  38   6   1   5   4   4]
 [  7   5  28   6   2  12  15]
 [ 22   6   7  13   4  14   8]
 [  9   9   6   8  52   2  16]
 [ 16   2   4   7   5  63   9]
 [ 27   9   9  11  11   8 115]]


In [13]:
path_model = 'cls_carModel.pkl'
pickle.dump(clf,open(path_model,'wb'))